# Preamble

In [1]:
from IPython.display import display, Markdown, HTML

# Make connection to Slack

In [2]:
from private_config import SLACK_TOKEN

In [3]:
from slackclient import SlackClient

sc = SlackClient(SLACK_TOKEN)

In [4]:
channel_info = sc.api_call("channels.list")
channels = channel_info['channels']

In [5]:
[(k['name'], k['id']) for k in channels]

[(u'building_comments', u'C6GQ4GWMT'),
 (u'building-comments', u'C6GKKF7U2'),
 (u'cafe_comments', u'C6FCGM656'),
 (u'external_relations', u'C2CMBSDG8'),
 (u'general', u'C2CLS7MHC'),
 (u'movequestions', u'C3WHBE9JN'),
 (u'random', u'C2CM9H9BL'),
 (u'theatreclub', u'C2CLV66QG')]

In [6]:
users = sc.api_call("users.list")
users_map = dict([(x['id'], x['name']) for x in users['members']])

# Get messages of interest

(i.e. building_comments and cafe_comments from actual users)

In [7]:
included_channels = [u'building_comments', u'cafe_comments']
channel_ids = dict([(k['name'], k['id']) for k in channels if k['name'] in included_channels])
channel_ids

{u'building_comments': u'C6GQ4GWMT', u'cafe_comments': u'C6FCGM656'}

In [8]:
channel_comments = {}

for cid in channel_ids:
    print "grabbing history for %s..." % cid
    # grab the history for each
    results = sc.api_call("channels.history", channel=channel_ids[cid], count=1000)
    channel_comments[cid] = results['messages']

grabbing history for building_comments...
grabbing history for cafe_comments...


In [131]:
# this transformed version is used elsewhere
chat_messages = [
    msg for msg in reversed(channel_comments['cafe_comments'])
    if msg['type'] == 'message' and 'subtype' not in msg and 'thread_ts' not in msg
]

In [132]:
chat_messages

[{u'text': u'<@U58RD27SN> ',
  u'ts': u'1501523716.028220',
  u'type': u'message',
  u'user': u'U4270RFH8'},
 {u'text': u'Hey ',
  u'ts': u'1501540563.446092',
  u'type': u'message',
  u'user': u'U4270RFH8'},
 {u'text': u'<@U61K5FHQU> <@U58RD27SN> ',
  u'ts': u'1501542130.920502',
  u'type': u'message',
  u'user': u'U4270RFH8'},
 {u'text': u'White bean soup was great',
  u'ts': u'1501544428.546795',
  u'type': u'message',
  u'user': u'U61K5FHQU'},
 {u'text': u'Fresh peach was not edible ',
  u'ts': u'1501544451.552736',
  u'type': u'message',
  u'user': u'U61K5FHQU'},
 {u'text': u'Double espresso was perfect',
  u'ts': u'1501544461.555391',
  u'type': u'message',
  u'user': u'U61K5FHQU'},
 {u'reactions': [{u'count': 1, u'name': u'smile', u'users': [u'U2DULB5AA']}],
  u'text': u'2nd day in a row of perfect espresso',
  u'ts': u'1501607376.356084',
  u'type': u'message',
  u'user': u'U61K5FHQU'},
 {u'text': u'Studybot campus-survey',
  u'ts': u'1501607451.399049',
  u'type': u'message',


# Aside: Emoji Rendering

In [10]:
import json
with open("./node_modules/emoji-datasource-apple/emoji.json") as fp:
    emoji = json.load(fp)

In [11]:
shortname_to_emoji = dict([(x['short_name'], x) for x in emoji])
shortname_to_emoji['pray']

{u'added_in': u'6.0',
 u'au': u'EAD2',
 u'category': u'People',
 u'docomo': None,
 u'google': u'FE35B',
 u'has_img_apple': True,
 u'has_img_emojione': True,
 u'has_img_facebook': True,
 u'has_img_google': True,
 u'has_img_messenger': True,
 u'has_img_twitter': True,
 u'image': u'1f64f.png',
 u'name': u'PERSON WITH FOLDED HANDS',
 u'sheet_x': 25,
 u'sheet_y': 0,
 u'short_name': u'pray',
 u'short_names': [u'pray'],
 u'skin_variations': {u'1F3FB': {u'added_in': u'8.0',
   u'has_img_apple': True,
   u'has_img_emojione': True,
   u'has_img_facebook': True,
   u'has_img_google': True,
   u'has_img_messenger': True,
   u'has_img_twitter': True,
   u'image': u'1f64f-1f3fb.png',
   u'sheet_x': 25,
   u'sheet_y': 1,
   u'unified': u'1F64F-1F3FB'},
  u'1F3FC': {u'added_in': u'8.0',
   u'has_img_apple': True,
   u'has_img_emojione': True,
   u'has_img_facebook': True,
   u'has_img_google': True,
   u'has_img_messenger': True,
   u'has_img_twitter': True,
   u'image': u'1f64f-1f3fc.png',
   u'sheet

In [100]:
# check skin tones
shortname_to_emoji['skin-tone-5']

{u'added_in': u'8.0',
 u'au': None,
 u'category': u'Skin Tones',
 u'docomo': None,
 u'google': None,
 u'has_img_apple': True,
 u'has_img_emojione': True,
 u'has_img_facebook': True,
 u'has_img_google': True,
 u'has_img_messenger': False,
 u'has_img_twitter': True,
 u'image': u'1f3fe.png',
 u'name': u'EMOJI MODIFIER FITZPATRICK TYPE-5',
 u'sheet_x': 10,
 u'sheet_y': 23,
 u'short_name': u'skin-tone-5',
 u'short_names': [u'skin-tone-5'],
 u'softbank': None,
 u'sort_order': 4,
 u'text': None,
 u'texts': None,
 u'unified': u'1F3FE',
 u'variations': []}

In [12]:
# approach 1. use slack's CDN and html
choice = shortname_to_emoji['pray']

# old percentages for pray icon 72.5% 27.5%
display(HTML("""
<span class="emoji-outer emoji-sizer"
style="width: 32px; height: 32px; display: block; background: url(https://a.slack-edge.com/bfaba/img/emoji_2016_06_08/sheet_apple_64_indexed_256colors.png);background-position: 72.5% 27.5%;background-size:4100%"
title="pray"></span>
"""))

In [125]:
# approach 2. use emojistatic's emojifont
display(HTML("""
<link href="https://emojistatic.github.io/emojifont/emojifont.min.css" rel="stylesheet">
<span alt="pray" style="font-size: 24px;">&#x%s&#x%s</span>
""" % (shortname_to_emoji['+1']['unified'], shortname_to_emoji['skin-tone-5']['unified'])))

In [14]:
# approach 3. directly load single image file
emoji_path = "./node_modules/emoji-datasource-apple/img/apple/64"
choice = shortname_to_emoji['pray']
display(HTML("""
<span style="width: 32px; height: 32px; display: block; background: url('%(imgdir)s'); background-size: 32px;"
title="pray"></span>
""" % {
    'imgdir': '%s/%s' % (emoji_path, choice[u'image'])
}))

# Format and display messages

In [15]:
from itertools import groupby
from datetime import datetime
from pytz import timezone

In [16]:
def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

print custom_strftime('%B {S}, %Y', datetime.now())

August 5th, 2017


In [17]:
formatted_chats = [
    {
        'datetime': (datetime
        .fromtimestamp(float(msg['ts']), timezone('UTC'))
        .astimezone(timezone('US/Eastern'))),
        'text': msg['text']
    }
    for msg in chat_messages
]

In [18]:
def escape_markdown(text):
    return text.replace("*", "\*")

In [19]:
["%s: %s" % (custom_strftime("%A, %B {S}, %-I:%M%p", msg['datetime']), msg['text']) for msg in formatted_chats]

[u'Monday, July 31st, 1:55PM: <@U58RD27SN> ',
 u'Monday, July 31st, 6:36PM: Hey ',
 u'Monday, July 31st, 7:02PM: <@U61K5FHQU> <@U58RD27SN> ',
 u'Monday, July 31st, 7:40PM: White bean soup was great',
 u'Monday, July 31st, 7:40PM: Fresh peach was not edible ',
 u'Monday, July 31st, 7:41PM: Double espresso was perfect',
 u'Tuesday, August 1st, 1:09PM: 2nd day in a row of perfect espresso',
 u'Tuesday, August 1st, 1:10PM: Studybot campus-survey',
 u'Tuesday, August 1st, 1:11PM: studybot campus-survey',
 u'Tuesday, August 1st, 2:26PM: Had a sandwich: turkey, mozzarella, all the veggies on multigrain.  It was very good.',
 u'Wednesday, August 2nd, 9:07AM: made to order omelette: (turkey, onions, and peppers): well-seasoned and perfectly cooked. (the _only_ thing is the cafe is still waiting on hot sauce..) but other than that, glowing reviews for breakfast!',
 u'Wednesday, August 2nd, 9:10AM: :+1::skin-tone-5:',
 u'Wednesday, August 2nd, 9:40AM: STARR should look into their credit card proc

In [20]:
grouped_messages = groupby(formatted_chats, lambda x: x['datetime'].strftime("%A, %B %-m"))
for date in (grouped_messages):
    print date[0], list(date[1])

Monday, July 7 [{'text': u'<@U58RD27SN> ', 'datetime': datetime.datetime(2017, 7, 31, 13, 55, 16, 28220, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Hey ', 'datetime': datetime.datetime(2017, 7, 31, 18, 36, 3, 446092, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'<@U61K5FHQU> <@U58RD27SN> ', 'datetime': datetime.datetime(2017, 7, 31, 19, 2, 10, 920502, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'White bean soup was great', 'datetime': datetime.datetime(2017, 7, 31, 19, 40, 28, 546795, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Fresh peach was not edible ', 'datetime': datetime.datetime(2017, 7, 31, 19, 40, 51, 552736, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Double espresso was perfect', 'datetime': datetime.datetime(2017, 7, 31, 19, 41, 1, 555391, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}]
Tuesday, August 8 [{'text': u'2nd day in a row of 

In [166]:
# emit channel messages as markdown
import re

find_names_re = re.compile(r'<@([^>]+)>')
find_emoji_re = re.compile(r':(?P<sn>[^:]+):(:(?P<mod>[^:]+):)?')

def replace_name(m):
    try:
        return "**@%s**" % users_map[m.group(1)]
    except KeyError:
        return "*%s*" % m.group(0)
    
def replace_emoji(m):
    try:
        emoji = shortname_to_emoji[m.group('sn')]
        
        if m.group('mod') and 'skin_variations' in emoji:
            # just print the emoji + variation out as two separate characters
            mod = shortname_to_emoji[m.group('mod')]
            # variation = emoji['skin_variations'][mod['unified']]
            return "&#x%s;&#x%s;" % (emoji['unified'], mod['unified'])
        else:
            return "&#x%s;" % emoji['unified']
    except KeyError:
        return m.group(0)

In [117]:
display(HTML(find_emoji_re.sub(replace_emoji, "i love <@U234> pizza :pray::skin-tone-5: hey")))

In [ ]:
# gathers output for emission
emissions = []
def emit(mkdown):
    global emissions
    emissions.append(mkdown)

emit(HTML("""<center><h1>Cornell Tech Weekly Digest</h1></center>"""))
# emit(HTML("""<link href="https://emojistatic.github.io/emojifont/emojifont.min.css" rel="stylesheet">"""))

# actually iterate over the messages now
for channel_name, comments in channel_comments.items():
    personal_messages = [
        msg for msg in reversed(comments)
        # if msg['type'] == 'message' and 'subtype' not in msg
        if msg['type'] == 'message' and 'subtype' not in msg and 'thread_ts' not in msg
    ]
    localized_chats = [
        {
            'datetime': (datetime
            .fromtimestamp(float(msg['ts']), timezone('UTC'))
            .astimezone(timezone('US/Eastern'))),
            'text': msg['text'],
            'user': msg['user']
        }
        for msg in personal_messages
    ]
    
    # actually print stuff now
    
    emit(Markdown("## \#%s" % channel_name))

    for day, messages in groupby(localized_chats, lambda x: custom_strftime("%A, %B {S}", x['datetime'])):
        emit(Markdown("### %s" % day))
        
        complete = []
        for message in messages:
            final_text = find_names_re.sub(replace_name, message['text'])
            final_text = find_emoji_re.sub(replace_emoji, final_text)
            
            data = {
                'text': final_text,
                'timeofday': custom_strftime("%-I:%M %p", message['datetime']),
                'author': users_map[message['user']]
            }
            complete.append("- *%(timeofday)s* (@%(author)s): %(text)s" % data)
        
        emit(Markdown("\n".join(complete)))

# ...and final processing/display

for em in emissions:
    display(em)

In [150]:
[min(y['ts'] for y in x) for x in channel_comments.values()]

[u'1501611741.429415', u'1501523689.012042']

In [161]:
# compute bounds
start_time = min(min(float(msg['ts']) for msg in messages) for messages in channel_comments.values())
end_time = max(max(float(msg['ts']) for msg in messages) for messages in channel_comments.values())

print start_time, end_time

def localize_ts(ts, tzone='US/Eastern'):
    return (datetime
            .fromtimestamp(float(ts), timezone('UTC'))
            .astimezone(timezone(tzone)))

start_datetime, end_datetime = localize_ts(start_time), localize_ts(end_time)

1501523689.01 1501880869.9


In [203]:
# html version

def replace_name_html(m):
    try:
        return '<b style="color: #3393cc;">@%s</b>' % users_map[m.group(1)]
    except KeyError:
        return "<b>%s</b>" % m.group(0)

# gathers output for emission
emissions = []
def emit(mkdown):
    global emissions
    emissions.append(mkdown)

emit(HTML("""<center>
<h1 style="margin-bottom: 0.5em;">Cornell Tech Weekly Digest</h1>
<div style="font-size: large; line-height: 150%%; color: #777; text-align: center;">messages from %(channel_list)s<br />from %(start_datestr)s to %(end_datestr)s</div>
</center>""" % {
    'channel_list': ", ".join([('<b style="color: black;">#%s</b>' % name) for name in channel_comments]),
    'start_datestr': '<b style="color: black;">%s</b>' % custom_strftime("%A, %B {S}, %Y", start_datetime),
    'end_datestr': '<b style="color: black;">%s</b>' % custom_strftime("%A, %B {S}, %Y", end_datetime)
}))

# actually iterate over the messages now
for channel_name, comments in channel_comments.items():
    personal_messages = [
        msg for msg in reversed(comments)
        # if msg['type'] == 'message' and 'subtype' not in msg
        if msg['type'] == 'message' and 'subtype' not in msg and 'thread_ts' not in msg
    ]
    localized_chats = [
        {
            'datetime': (datetime
            .fromtimestamp(float(msg['ts']), timezone('UTC'))
            .astimezone(timezone('US/Eastern'))),
            'text': msg['text'],
            'user': msg['user']
        }
        for msg in personal_messages
    ]
    
    # actually print stuff now
    
    emit(HTML('<h2 style="font-size: x-large;">#%s</h2>' % channel_name))

    for day, messages in groupby(localized_chats, lambda x: custom_strftime("%A, %B {S}", x['datetime'])):
        emit(HTML('<h3 style="color: #555;">%s</h3>' % day))
        emit(HTML('<ul style="margin-bottom: 1em;">'))
        
        for message in messages:
            final_text = find_names_re.sub(replace_name_html, message['text'])
            final_text = find_emoji_re.sub(replace_emoji, final_text)
            
            data = {
                'text': final_text,
                'timeofday': custom_strftime("%-I:%M %p", message['datetime']),
                'author': users_map[message['user']]
            }
            emit(HTML("""<li style="margin-bottom: 0.25em;"><b style="font-size: larger;">%(author)s</b> <span style="color: #777; font-size: smaller;">%(timeofday)s</span><br />%(text)s</li>""" % data))
        
        emit(HTML("</ul>"))
        
# ...and final processing/display

for em in emissions:
    display(em)

# Email result

In [170]:
import yagmail
yag = yagmail.SMTP('falquaddoomi')

In [201]:
from markdown import markdown
html = ""

for em in emissions:
    if type(em) == Markdown:
        html += markdown(em.data, safe_mode='escape')
    else:
        html += em.data

# html = "".join([markdown(x.data, safe_mode='escape') for x in emissions])
html = find_emoji_re.sub(replace_emoji, html)

yag.send('falquaddoomi@gmail.com', 'Cornell Tech Weekly Digest', html)

{}